In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

In [4]:
filepath = '../data/data_pre-imputing.csv'
data = pd.read_csv(filepath)

In [6]:
data.head()

FIPS    State   County  % Adults with Diabetes  % Completed High School  \
0  1001  Alabama  Autauga                    10.7                90.402608   
1  1003  Alabama  Baldwin                     9.8                91.599310   
2  1005  Alabama  Barbour                    15.6                76.492221   
3  1007  Alabama     Bibb                    12.4                78.863422   
4  1009  Alabama   Blount                    10.5                81.755151   

   School Funding Adequacy  % Non-Hispanic White  % Voter Turnout  \
0               -3606.7820             72.051072        66.182078   
1                -537.1533             82.978473        65.290949   
2              -23626.5800             45.272403        54.021572   
3               -6970.6460             73.996819        54.563548   
4               -2788.9990             85.992741        64.187994   

   Food Environment Index  Median Household Income  ...    % Black  \
0                     6.7                  70148.0  ...  21.011061   
1                     7.5                  71704.0  ...   8.224887   
2                     6.0                  41151.0  ...  47.195013   
3                     7.6                  54309.0  ...  20.736196   
4                     7.7                  60553.0  ...   1.727383   

   % Adults Reporting Currently Smoking  % Frequent Mental Distress  \
0                                  16.9                        18.0   
1                                  15.0                        17.3   
2                                  25.0                        20.5   
3                                  22.0                        19.4   
4                                  19.6                        18.8   

   % Adults with Obesity  % Children in Single-Parent Households  \
0                   38.9                               22.702506   
1                   37.2                               19.051287   
2                   43.4                               50.660451   
3                   39.6                               30.713370   
4                   37.7                               22.969477   

   % Insufficient Sleep  % Food Insecure  % Children in Poverty  \
0                  37.5             13.3                   15.7   
1                  35.1             11.8                   16.1   
2                  43.9             17.8                   37.7   
3                  40.0             14.9                   25.5   
4                  37.3             13.7                   15.8   

   % Frequent Physical Distress  % Physically Inactive  
0                          11.7                   29.1  
1                          11.0                   27.6  
2                          15.8                   40.2  
3                          14.0                   36.0  
4                          12.8                   30.4  

[5 rows x 32 columns]

In [5]:
data_drop_null_state_features = data.drop(columns=['% Voter Turnout', 'School Funding Adequacy']) #we may not even need to drop these anymore

In [8]:
len(data_drop_null_state_features.columns)

30

In [6]:
features = data.columns[4:]
target = '% Adults with Diabetes'

In [12]:
data_train, data_test = train_test_split(data,
                                                    shuffle=True,
                                                    random_state=42,
                                                    test_size=0.2)

In [13]:
xgb_pipe = Pipeline([('impute', KNNImputer()),
                     ('scale', StandardScaler()),
                     ('xgb', XGBRegressor())])

X_train = data_train[features]
y_train = data_train[target]

xgb_pipe.fit(X_train, y_train)

Pipeline(steps=[('impute', KNNImputer()), ('scale', StandardScaler()),
                ('xgb',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [14]:
from sklearn.metrics import root_mean_squared_error

xgb_train_preds = xgb_pipe.predict(X_train)

root_mean_squared_error(y_train, xgb_train_preds)

np.float64(0.042267273818276656)